# Data loading 
Next cells are used to load the data we need.

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1 
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import sys
import matplotlib.pyplot as plt
sys.path.append('../../../')


In [2]:
#data which contains users (row), items(col) and implicit interaction (data)
dataset = pd.read_csv('../../data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [3]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [4]:
ICM_df = pd.read_csv('../../data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [5]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [6]:
grouped_users = dataset.groupby(['row']).count()
sorted_users = grouped_users.sort_values(by=['col'], ascending=True)
sorted_users = sorted_users.index.to_numpy()
block_size = int(len(sorted_users)*0.75)

start_pos_group_0 = 0*block_size
end_pos_group_0 = min((0+1)*block_size, len(sorted_users))
users_in_group_0 = sorted_users[start_pos_group_0:end_pos_group_0]
users_not_in_group_0_flag = np.isin(sorted_users, users_in_group_0, invert = True)
users_not_in_group_0 = sorted_users[users_not_in_group_0_flag]


start_pos_group_1 = 1*block_size
end_pos_group_1 = min((1+1)*block_size, len(sorted_users))
users_in_group_1 = sorted_users[start_pos_group_1:end_pos_group_1]
users_not_in_group_1_flag = np.isin(sorted_users, users_in_group_1, invert = True)
users_not_in_group_1 = sorted_users[users_not_in_group_1_flag]

In [7]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Base.Evaluation.Evaluator import EvaluatorHoldout

URM_train_1, URM_validation_1 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 1)

URM_train_2, URM_validation_2 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 2)

URM_train_3, URM_validation_3 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 3)

URM_train_4, URM_validation_4 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 4)

URM_train_5, URM_validation_5 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 5)

URM_train_vector = [URM_train_1,URM_train_2,URM_train_3,URM_train_4,URM_train_5]

evaluator_validation_1 = EvaluatorHoldout(URM_validation_1, cutoff_list=[10], ignore_users = users_not_in_group_1)
evaluator_validation_2 = EvaluatorHoldout(URM_validation_2, cutoff_list=[10], ignore_users = users_not_in_group_1)
evaluator_validation_3 = EvaluatorHoldout(URM_validation_3, cutoff_list=[10], ignore_users = users_not_in_group_1)
evaluator_validation_4 = EvaluatorHoldout(URM_validation_4, cutoff_list=[10], ignore_users = users_not_in_group_1)
evaluator_validation_5 = EvaluatorHoldout(URM_validation_5, cutoff_list=[10], ignore_users = users_not_in_group_1)

Evaluator_vector = [evaluator_validation_1, evaluator_validation_2, evaluator_validation_3, evaluator_validation_4, evaluator_validation_5]


EvaluatorHoldout: Ignoring 5960 Users
EvaluatorHoldout: Ignoring 5960 Users
EvaluatorHoldout: Ignoring 5960 Users
EvaluatorHoldout: Ignoring 5960 Users
EvaluatorHoldout: Ignoring 5960 Users


In [8]:
from MatrixFactorization.IALSRecommender_implicit import IALSRecommender_implicit
i=1
for URM_train, evaluator in zip(URM_train_vector,Evaluator_vector):
    
    URM_train_ICM_all_IALS = sps.vstack([URM_train*(1-0.6790082920718808), ICM_all.T*0.6790082920718808])
    recommender = IALSRecommender_implicit(URM_train_ICM_all_IALS)
    recommender.fit(n_factors = 736, regularization = 0.4021244259692904, iterations=77)

    #recommender.URM_train = URM_train.tocsr()
    
    result_dict, _ = evaluator.evaluateRecommender(recommender)
    print(result_dict)
    
    filename = 'MF_IALS_fold_number' + str(i)
    recommender.save_model('MF_saved_group_1/', file_name = filename)
    i = i+1

Recommender_Base_Class: URM Detected 73 (0.26 %) cold users.
Recommender_Base_Class: URM Detected 1 (0.00 %) cold items.



EvaluatorHoldout: Processed 1951 ( 100.00% ) in 2.85 sec. Users per second: 686
{10: {'ROC_AUC': 0.31220359115797436, 'PRECISION': 0.08421322398769765, 'PRECISION_RECALL_MIN_DEN': 0.1529882925321159, 'RECALL': 0.13789806398702084, 'MAP': 0.06910877325283228, 'MRR': 0.24331335985615823, 'NDCG': 0.12118714756123733, 'F1': 0.10456776560883897, 'HIT_RATE': 0.8421322398769862, 'ARHR': 0.30804735870087024, 'NOVELTY': 0.0053431273378307005, 'AVERAGE_POPULARITY': 0.12373629465380079, 'DIVERSITY_MEAN_INTER_LIST': 0.993622783845234, 'DIVERSITY_HERFINDAHL': 0.9993113494873503, 'COVERAGE_ITEM': 0.2557844080846968, 'COVERAGE_ITEM_CORRECT': 0.04604427333974976, 'COVERAGE_USER': 0.9818822345244087, 'COVERAGE_USER_CORRECT': 0.5027679919476598, 'DIVERSITY_GINI': 0.11697165320919978, 'SHANNON_ENTROPY': 11.672104816047707}}
Recommender_Base_Class: Saving model in file 'MF_saved_group_1/MF_IALS_fold_number1'
Recommender_Base_Class: Saving complete
Recommender_Base_Class: URM Detected 59 (0.21 %) cold use


EvaluatorHoldout: Processed 1947 ( 100.00% ) in 2.81 sec. Users per second: 694
{10: {'ROC_AUC': 0.3114177692991257, 'PRECISION': 0.08320493066255685, 'PRECISION_RECALL_MIN_DEN': 0.15275800784275348, 'RECALL': 0.13841440391818155, 'MAP': 0.0716607309143292, 'MRR': 0.24767896886540922, 'NDCG': 0.12429313316202709, 'F1': 0.10393281707572088, 'HIT_RATE': 0.8320493066255779, 'ARHR': 0.3104960011739679, 'NOVELTY': 0.005348186781905363, 'AVERAGE_POPULARITY': 0.12205399611442375, 'DIVERSITY_MEAN_INTER_LIST': 0.9938301263017761, 'DIVERSITY_HERFINDAHL': 0.9993319684531718, 'COVERAGE_ITEM': 0.2575553416746872, 'COVERAGE_ITEM_CORRECT': 0.04600577478344562, 'COVERAGE_USER': 0.9798691494715652, 'COVERAGE_USER_CORRECT': 0.5012581781580272, 'DIVERSITY_GINI': 0.11848192531717872, 'SHANNON_ENTROPY': 11.697619413138941}}
Recommender_Base_Class: Saving model in file 'MF_saved_group_1/MF_IALS_fold_number2'
Recommender_Base_Class: Saving complete
Recommender_Base_Class: URM Detected 73 (0.26 %) cold users


EvaluatorHoldout: Processed 1961 ( 100.00% ) in 2.81 sec. Users per second: 698
{10: {'ROC_AUC': 0.30760336482034645, 'PRECISION': 0.08184599694033544, 'PRECISION_RECALL_MIN_DEN': 0.15165185401034442, 'RECALL': 0.1376335756297327, 'MAP': 0.06845682505442163, 'MRR': 0.2378169948924663, 'NDCG': 0.11999266567102869, 'F1': 0.10264970974720021, 'HIT_RATE': 0.8184599694033656, 'ARHR': 0.3010461944424209, 'NOVELTY': 0.005343381594831683, 'AVERAGE_POPULARITY': 0.12274045789922461, 'DIVERSITY_MEAN_INTER_LIST': 0.9936491689995733, 'DIVERSITY_HERFINDAHL': 0.9993142463660971, 'COVERAGE_ITEM': 0.2558999037536092, 'COVERAGE_ITEM_CORRECT': 0.04519730510105871, 'COVERAGE_USER': 0.9869149471565174, 'COVERAGE_USER_CORRECT': 0.4972320080523402, 'DIVERSITY_GINI': 0.11711503357667306, 'SHANNON_ENTROPY': 11.675005057564167}}
Recommender_Base_Class: Saving model in file 'MF_saved_group_1/MF_IALS_fold_number3'
Recommender_Base_Class: Saving complete
Recommender_Base_Class: URM Detected 76 (0.27 %) cold users


EvaluatorHoldout: Processed 1957 ( 100.00% ) in 2.81 sec. Users per second: 697
{10: {'ROC_AUC': 0.3092969275940665, 'PRECISION': 0.08339294839039256, 'PRECISION_RECALL_MIN_DEN': 0.15254175081717203, 'RECALL': 0.13861761431825637, 'MAP': 0.07217427590654342, 'MRR': 0.24383998020942288, 'NDCG': 0.1235749153225213, 'F1': 0.10413677093383952, 'HIT_RATE': 0.8339294839039346, 'ARHR': 0.3115146279939333, 'NOVELTY': 0.005344643805884395, 'AVERAGE_POPULARITY': 0.12170268889226475, 'DIVERSITY_MEAN_INTER_LIST': 0.9936867602325249, 'DIVERSITY_HERFINDAHL': 0.9993179000007573, 'COVERAGE_ITEM': 0.25528392685274304, 'COVERAGE_ITEM_CORRECT': 0.04585178055822907, 'COVERAGE_USER': 0.9849018621036739, 'COVERAGE_USER_CORRECT': 0.5007549068948163, 'DIVERSITY_GINI': 0.116489620779623, 'SHANNON_ENTROPY': 11.671418247964164}}
Recommender_Base_Class: Saving model in file 'MF_saved_group_1/MF_IALS_fold_number4'
Recommender_Base_Class: Saving complete
Recommender_Base_Class: URM Detected 66 (0.24 %) cold users.


EvaluatorHoldout: Processed 1950 ( 100.00% ) in 2.80 sec. Users per second: 696
{10: {'ROC_AUC': 0.301503988603989, 'PRECISION': 0.08338461538461435, 'PRECISION_RECALL_MIN_DEN': 0.1513229548229544, 'RECALL': 0.1371499371319883, 'MAP': 0.06958571953472754, 'MRR': 0.24197822547822528, 'NDCG': 0.12189537996330704, 'F1': 0.10371340569785703, 'HIT_RATE': 0.8338461538461538, 'ARHR': 0.3050877085877091, 'NOVELTY': 0.0053479497734922376, 'AVERAGE_POPULARITY': 0.12188573021181695, 'DIVERSITY_MEAN_INTER_LIST': 0.9938835168594019, 'DIVERSITY_HERFINDAHL': 0.9993373833004602, 'COVERAGE_ITEM': 0.2599422521655438, 'COVERAGE_ITEM_CORRECT': 0.04531280076997113, 'COVERAGE_USER': 0.9813789632611978, 'COVERAGE_USER_CORRECT': 0.4967287367891293, 'DIVERSITY_GINI': 0.11996530130376759, 'SHANNON_ENTROPY': 11.713246979049728}}
Recommender_Base_Class: Saving model in file 'MF_saved_group_1/MF_IALS_fold_number5'
Recommender_Base_Class: Saving complete
